This Notebook will be mainly used for Capstone Project

In [5]:
import pandas as pd
import numpy as np

In [6]:
print("Hello Capstone Project Course!")

Hello Capstone Project Course!


In [7]:
import json 
!conda install -c anaconda seaborn -y
from geopy.geocoders import Nominatim 
import folium 

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - seaborn


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openssl-1.1.1              |       h7b6447c_0         5.0 MB  anaconda
    ca-certificates-2020.1.1   |                0         132 KB  anaconda
    seaborn-0.9.0              |     pyh91ea838_1         164 KB  anaconda
    certifi-2019.11.28         |           py36_0         156 KB  anaconda
    ------------------------------------------------------------
                                           Total:         5.5 MB

The following packages will be UPDATED:

    ca-certificates: 2019.11.27-0       --> 2020.1.1-0         anaconda
    certifi:         2019.11.28-py36_0  --> 2019.11.28-py36_0  anaconda
    openssl:         1.1.1d-h7b6447c_3  --> 1.1.1-h7b6447c_0   anaconda
    seaborn:         0.9.0-py

ModuleNotFoundError: No module named 'folium'

The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood

In [18]:
tables = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M', header=0)

# I did not use Beatiful Soup for this as I felt this would be easier and faster
cols = ['Postcode', 'Borough','Neighbourhood']

# Note I converted the boolean output of array_equal function to String as it 
# wasn't working as expected when used in the if condition as expected
for table in tables:
    if(str(np.array_equal(np.array(table.columns),np.array(cols)))=="True"):
        postal_df = pd.DataFrame(table)    
    break


Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

In [19]:
postal_df = postal_df[postal_df.Borough!="Not assigned"]
print("Dataframe shape: ",postal_df.shape)
postal_df.head()

Dataframe shape:  (210, 3)


,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor


If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.

In [20]:
postal_df['NewNeighbourhood'] = np.where(postal_df['Neighbourhood']=='Not assigned',postal_df['Borough'],postal_df['Neighbourhood'])
postal_df.head(9)

,Postcode,Borough,Neighbourhood,NewNeighbourhood
2,M3A,North York,Parkwoods,Parkwoods
3,M4A,North York,Victoria Village,Victoria Village
4,M5A,Downtown Toronto,Harbourfront,Harbourfront
5,M6A,North York,Lawrence Heights,Lawrence Heights
6,M6A,North York,Lawrence Manor,Lawrence Manor
7,M7A,Downtown Toronto,Queen's Park,Queen's Park
9,M9A,Queen's Park,Not assigned,Queen's Park
10,M1B,Scarborough,Rouge,Rouge
11,M1B,Scarborough,Malvern,Malvern


More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma.

In [21]:
postal_code_df = pd.DataFrame(postal_df.groupby(['Postcode','Borough'])['NewNeighbourhood'].apply(','.join).reset_index())
postal_code_df = postal_code_df.rename(columns = {"Postcode": "PostalCode","NewNeighbourhood":"Neighborhood"})
print(postal_code_df.head())
print("Dataframe shape:",postal_code_df.shape)

  PostalCode      Borough                          Neighborhood
0        M1B  Scarborough                         Rouge,Malvern
1        M1C  Scarborough  Highland Creek,Rouge Hill,Port Union
2        M1E  Scarborough       Guildwood,Morningside,West Hill
3        M1G  Scarborough                                Woburn
4        M1H  Scarborough                             Cedarbrae
Dataframe shape: (103, 3)


latitude and the longitude coordinates of each neighborhood.

In [22]:
!pip install geocoder

     |████████████████████████████████| 102kB 7.3MB/s ta 0:00:011


In [23]:
import geocoder

In [24]:
!wget -q -O "toronto_coordinates.csv" http://cocl.us/Geospatial_data
coordinates = pd.read_csv('toronto_coordinates.csv')

In [35]:
coordinates.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [45]:
toronto_df_1 = postal_code_df.set_index('PostalCode')
coordiantes_1 = coordinates.set_index('Postal Code')
toronto_df_coordinates = pd.concat([toronto_df_1, coordinates_1], axis=1, join='inner')
toronto_df_coordinates.index.name = 'PostalCode'
toronto_df_coordinates.reset_index(inplace=True)

print('dataframe shape:',toronto_df_coordinates.shape)
toronto_df_coordinates.head()

dataframe shape: (103, 5)


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
